https://www.kaggle.com/gogo827jz/inference-demo-notebook

# Code Section

# Sandbox Section

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/optiver-realized-volatility-prediction/sample_submission.csv
/kaggle/input/optiver-realized-volatility-prediction/train.csv
/kaggle/input/optiver-realized-volatility-prediction/test.csv
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=97/888f813404d8417ca8d6b8aebd5f2951.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=43/bb0efa57f511470e817880842e3e2afa.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=21/1d8dc18ebfee47ffbb54b04e6afc0634.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=72/60f62a03d8854605901dda072c84db39.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=4/761268d671f9429abb29d9d2895e9bd2.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=112/cd283097a5b54293ba400a19e811a7f9.parquet
/kaggle/input/optiver-realized-volatility-pr

In [16]:
import pandas as pd
pd.set_option('display.width', 1000)
df = pd.read_csv('../input//optiver-realized-volatility-prediction/sample_submission.csv')
df.head()

,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


In [3]:
df = pd.read_csv('../input//optiver-realized-volatility-prediction/train.csv')
df.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [4]:
df = pd.read_csv('../input//optiver-realized-volatility-prediction/test.csv')
df.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [5]:
import glob
list_book_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
print(list_book_test)

['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0']


In [6]:
pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0/7832c05caae3489cbcbbb9b02cf61711.parquet', engine='pyarrow')

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,4,0,1.000049,1.000590,0.999656,1.000639,91,100,100,24
1,4,1,1.000049,1.000590,0.999656,1.000639,91,100,100,20
2,4,5,1.000049,1.000639,0.999656,1.000885,290,20,101,15


In [7]:
list_trade_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/*')
print(list_trade_test)

['/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0']


In [8]:
pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0/31c83a67d81349208e7d5eace9dbbac8.parquet', engine='pyarrow')

,time_id,seconds_in_bucket,price,size,order_count
0,4,7,1.000344,1,1
1,4,24,1.000049,100,7
2,4,27,1.000059,100,3


# <font size = 4 color = 'yellow'> Reading the book_test file. PS : book_test is provided for stock_id = 0 only.

In [9]:
list_order_book_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')
print(f"list_order_book_file_test \t\t: {list_order_book_file_test}, \ntype(list_order_book_file_test) \t: {type(list_order_book_file_test)}")

list_order_book_file_test = sorted(list_order_book_file_test, key = lambda x: x.split('=')[1])
print(f"list_order_book_file_test \t\t:Sorted {list_order_book_file_test}, \ntype(list_order_book_file_test) \t: {type(list_order_book_file_test)}")

list_order_book_file_test 		: ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], 
type(list_order_book_file_test) 	: <class 'list'>
list_order_book_file_test 		:Sorted ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], 
type(list_order_book_file_test) 	: <class 'list'>


In [81]:
func_print = True

In [82]:
from tqdm.auto import tqdm
import numpy as np

def log_return(list_stock_prices):
    if func_print : print(f'\n\nInside function log_return \nParameters : list_stock_prices =\n{list_stock_prices} \n')
    
    ret_value = np.log(list_stock_prices).diff()
    if func_print : print(f'Returning from log_return. ret_value = np.log(list_stock_prices).diff() : \n{ret_value}')
    
    return np.log(list_stock_prices).diff()

def realized_volatility(series_log_return):
    if func_print : print(f'\n\nInside function realized_volatility \nParameters : series_log_return =\n {series_log_return} \n')
    ret_value = np.sqrt(np.sum(series_log_return ** 2))
    if func_print : print(f'Returning from function realized_volatility \nret_value = np.sqrt(np.sum(series_log_return ** 2)) : \nret_value = {ret_value} \n\n')
    return np.sqrt(np.sum(series_log_return ** 2))

def calculate_wap(df):
    if func_print : print(f'\n\nInside function calculate_wap \nParameters : df =\n {df} \n')
    a1 = df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']
    a2 = df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']
    b = df['bid_size1'] + df['ask_size1'] + df['bid_size2']+ df['ask_size2']
    if func_print : print(f'Return from function calculate_wap \n\n')
    return (a1 + a2)/ b

def realized_volatility_per_time_id(file_path, prediction_column_name):
    if func_print : print(f'\n\nInside function realized_volatility_per_time_id \nParameters : file_path = {file_path}, prediction_column_name: {prediction_column_name} \n')
    
    df_book_data = pd.read_parquet(file_path)
    if func_print : print(f'df_book_data before calculate_wap \n: {df_book_data} \n')
    
    df_book_data['wap'] = calculate_wap(df_book_data)
    if func_print : print(f'df_book_data after calculate_wap \n: {df_book_data} \n')
    
    df_book_data_grouped = df_book_data.groupby(['time_id'])['wap']
    
    if func_print : print('\nPrinting df_book_data.groupby([''time_id''])[''wap''] in a loop')
    for key, item in df_book_data_grouped:
        if func_print : print(f'key : {key}')
        if func_print : print(f'type(item) : {type(item)}')
        if func_print : print(f'item : {item} \n')
        #print(df_book_data_grouped.get_group(key))
    
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    if func_print : print(f'Back to function realized_volatility_per_time_id')
    if func_print : print(df_book_data)
    
    
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    if func_print : print('\ndf_book_data after removing null log_return rows')
    if func_print : print(df_book_data)
    
    if func_print : print('\nPrinting df_book_data.groupby([''time_id''])[''log_return''] in a loop')
    for key, item in df_book_data.groupby(['time_id'])['log_return']:
        if func_print : print(f'key : {key}')
        if func_print : print(f'type(item) : {type(item)}')
        if func_print : print(f'item : {item} \n')
    
    
    
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    if func_print : print(f'Back to function realized_volatility_per_time_id')
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock}')
    
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    if func_print : print(f'renamed column log_return. \ndf_realized_vol_per_stock :\n{df_realized_vol_per_stock} \n')
    
    stock_id = file_path.split('=')[1]
    if func_print : print(f'Evaluating stock_id = file_path.split(\'=\')[1]. \nfile_path : {file_path}\nstock_id : {stock_id} \n')
    
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock} \n')
    if func_print : print('Evaluating df_realized_vol_per_stock[\'row_id\'] = df_realized_vol_per_stock[\'time_id\'].apply(lambda x:f\'{stock_id}-{x}\')')
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    if func_print : print(f'df_realized_vol_per_stock :\n{df_realized_vol_per_stock}')
    
    ret_value = df_realized_vol_per_stock[['row_id', prediction_column_name]]
    if func_print : print('Returning from function realized_volatility_per_time_id.')
    if func_print : print('Return Expr : ret_value = df_realized_vol_per_stock[[\'row_id\', prediction_column_name]]')
    if func_print : print(f'Return Val :\n{ret_value}')
    return df_realized_vol_per_stock[['row_id', prediction_column_name]]

def past_realized_volatility_per_stock(list_file, prediction_column_name):
    if func_print : print(f'\n\nInside function past_realized_volatility_per_stock \nParameters : list_file = {list_file}, prediction_column_name: {prediction_column_name} ')
    df_past_realized = pd.DataFrame()
    
    for file in tqdm(list_file):
        if func_print : print(f'Inside Loop. for file in tqdm(list_file).\nlist_file : {list_file} \nfile : {file}')
        df_past_realized = pd.concat([df_past_realized, realized_volatility_per_time_id(file, prediction_column_name)])
        if func_print : print('Back to past_realized_volatility_per_stock')
        if func_print : print(f'df_past_realized :\n{df_past_realized}')
    if func_print : print(f'Returning from past_realized_volatility_per_stock. \nret_value :\n{df_past_realized}')
    return df_past_realized

# <font size = 4 color = 'yellow'> Calling function past_realized_volatility_per_stock

In [83]:
df_naive_pred_test = past_realized_volatility_per_stock(list_file = list_order_book_file_test, prediction_column_name = 'pred')



Inside function past_realized_volatility_per_stock 
Parameters : list_file = ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'], prediction_column_name: pred 


  0%|          | 0/1 [00:00<?, ?it/s]

Inside Loop. for file in tqdm(list_file).
list_file : ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0'] 
file : /kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0


Inside function realized_volatility_per_time_id 
Parameters : file_path = /kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0, prediction_column_name: pred 

df_book_data before calculate_wap 
:    time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  bid_size1  ask_size1  bid_size2  ask_size2
0        4                  0    1.000049    1.000590    0.999656    1.000639         91        100        100         24
1        4                  1    1.000049    1.000590    0.999656    1.000639         91        100        100         20
2        4                  5    1.000049    1.000639    0.999656    1.000885        290         20        101         15 



Inside function calculate_wap 
Parameters : df =
  

In [84]:
df_naive_pred_test

,row_id,pred
0,0-4,0.000263


In [85]:
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
test.head()

,stock_id,time_id,row_id
0,0,4,0-4
1,0,32,0-32
2,0,34,0-34


In [86]:
df_naive_pred_test.head(100)

,row_id,pred
0,0-4,0.000263


In [87]:
test = test['row_id'].to_frame()
test = test.merge(df_naive_pred_test[['row_id', 'pred']], on = ['row_id'], how = 'left')
test.head()

,row_id,pred
0,0-4,0.000263
1,0-32,NaN
2,0-34,NaN


In [88]:
n_splits = 5
xgb_preds = pd.DataFrame()
cols = ['seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'wap', 'log_return', 'pred']

In [89]:
list_order_trade_file_test = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/*')
list_order_trade_file_test = sorted(list_order_trade_file_test, key = lambda x: x.split('=')[1])

In [90]:
for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    print(i)
    print(b)
    print(t)

  0%|          | 0/1 [00:00<?, ?it/s]

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [91]:
print(f"list_order_book_file_test : {list_order_book_file_test}")
print(f"list_order_trade_file_test : {list_order_trade_file_test}")

list_order_book_file_test : ['/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0']
list_order_trade_file_test : ['/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0']


In [92]:
# printing as tuple
for i, m_tuple in enumerate(zip(list_order_book_file_test, list_order_trade_file_test)): 
    print(i)
    print(m_tuple)

0
('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0', '/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0')


In [93]:
# print as individual
for i, (b, t) in enumerate(zip(list_order_book_file_test, list_order_trade_file_test)): 
    print(i)
    print(b) #book
    print(t) #trade

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [94]:
# adding tqdm
for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    print(i)
    print(b)
    print(t)

  0%|          | 0/1 [00:00<?, ?it/s]

0
/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/stock_id=0
/kaggle/input/optiver-realized-volatility-prediction/trade_test.parquet/stock_id=0


In [95]:
# explaining assertion

assert 2 + 2 == 5, "Houston we've got a problem"

AssertionError: Houston we've got a problem

In [119]:
func_print = False

for i, (b, t) in tqdm(enumerate(zip(list_order_book_file_test, list_order_trade_file_test)), total = len(list_order_book_file_test)):
    
    assert b.split('=')[1] == t.split('=')[1]
    
    stock_id = b.split('=')[1]
    print(f"1. stock_id : {stock_id} \n")
    book_data = pd.read_parquet(b)
    print(f"2. book_data :\n{book_data} \n")
    trade_data = pd.read_parquet(t)
    print(f"3. trade_data :\n{trade_data} \n")
    
    book_data['row_id'] = book_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
    print(f"3. book_data :\n{book_data} \n")
    
    book_data['wap'] = calculate_wap(book_data)
    print(f"4. book_data :\n{book_data} \n")
    
    book_data['log_return'] = book_data.groupby(['time_id'])['wap'].apply(log_return).fillna(0)
    print(f"5. book_data :\n{book_data} \n")
    
    for trans in ['rank', 'cumcount', 'cummax', 'cummin', 'cumsum']:
        print('\n\n************************************************************************')
        print(f"\ntrans : {trans}")
        
        for feature in ['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2', 'wap', 'log_return']:
            feature_trans = feature + '_' + trans
            print(f"feature_trans : {feature_trans}")
            if i == 0:
                cols.append(feature_trans)
            book_data[feature_trans] = book_data.groupby('time_id')[feature].transform(trans)
            print(f"6. book_data[{feature_trans}] :\n{book_data[[feature, feature_trans]]} \n")
        
        for trans in ['mean', 'std', 'max', 'min']:
            for feature in ['price', 'size', 'order_count']:
                feature_trans = 'trade_' + feature + '_' + trans
                if i == 0:
                    cols.append(feature_trans)
                trade_data[feature_trans] = trade_data.groupby('time_id')[feature].transform(trans)
                print(f"7. trade_data[{feature_trans}] :\n{trade_data[[feature, feature_trans]]} \n")

                book_data = book_data.merge(trade_data[['time_id', feature_trans]].drop_duplicates(), on = 'time_id', how = 'left')
                print(f"8. book_data :\n{book_data.iloc[:, -1]} \n")

  0%|          | 0/1 [00:00<?, ?it/s]

1. stock_id : 0 

2. book_data :
   time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  bid_size1  ask_size1  bid_size2  ask_size2
0        4                  0    1.000049    1.000590    0.999656    1.000639         91        100        100         24
1        4                  1    1.000049    1.000590    0.999656    1.000639         91        100        100         20
2        4                  5    1.000049    1.000639    0.999656    1.000885        290         20        101         15 

3. trade_data :
   time_id  seconds_in_bucket     price  size  order_count
0        4                  7  1.000344     1            1
1        4                 24  1.000049   100            7
2        4                 27  1.000059   100            3 

3. book_data :
   time_id  seconds_in_bucket  bid_price1  ask_price1  bid_price2  ask_price2  bid_size1  ask_size1  bid_size2  ask_size2 row_id
0        4                  0    1.000049    1.000590    0.999656    1.000639  

In [122]:
print(f'test : {test.columns.values}')

test : ['row_id' 'pred']


In [121]:
book_data = book_data.merge(test, on = 'row_id', how = 'left')
print(f'book_data : {book_data.columns.values}')

book_data : ['time_id' 'seconds_in_bucket' 'bid_price1' 'ask_price1' 'bid_price2'
 'ask_price2' 'bid_size1' 'ask_size1' 'bid_size2' 'ask_size2' 'row_id'
 'wap' 'log_return' 'bid_price1_rank' 'ask_price1_rank' 'bid_price2_rank'
 'ask_price2_rank' 'bid_size1_rank' 'ask_size1_rank' 'bid_size2_rank'
 'ask_size2_rank' 'wap_rank' 'log_return_rank' 'trade_price_mean_x'
 'trade_size_mean_x' 'trade_order_count_mean_x' 'trade_price_std_x'
 'trade_size_std_x' 'trade_order_count_std_x' 'trade_price_max_x'
 'trade_size_max_x' 'trade_order_count_max_x' 'trade_price_min_x'
 'trade_size_min_x' 'trade_order_count_min_x' 'bid_price1_cumcount'
 'ask_price1_cumcount' 'bid_price2_cumcount' 'ask_price2_cumcount'
 'bid_size1_cumcount' 'ask_size1_cumcount' 'bid_size2_cumcount'
 'ask_size2_cumcount' 'wap_cumcount' 'log_return_cumcount'
 'trade_price_mean_y' 'trade_size_mean_y' 'trade_order_count_mean_y'
 'trade_price_std_y' 'trade_size_std_y' 'trade_order_count_std_y'
 'trade_price_max_y' 'trade_size_max_y' 't

In [123]:
print('test')

test
